## Forked from https://www.kaggle.com/code/abdurrafae/improved-code-interpretation

**Lewis:** the only changes in this notebook are those needed to run the original one with the new Kaggle evaluation API

Forked From  https://kaggle.com/code/xiaoz259/pure-rng/notebook

credits:
https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline \
https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama \
https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline

In [1]:
import time
NOTEBOOK_START_TIME = time.time()

PRIVATE = False
TRAIN_PATH = 'data/not5.csv'

In [2]:
TOTAL_TOKENS = 1024

TIME_LIMIT = 31500 if PRIVATE else 6600 * 2
PER_Q_TIME_LIMIT = 640

BEST_COUNT_THRESHOLD = 15   # it was set as np.sqrt(jj)

TEMP = 0.9
TEMP_CODE = 0.9

In [3]:
import pandas as pd
if not PRIVATE:
    class train_env():
        def __init__(self, randomize=False):
            self.randomlize = randomize
            
            self.df = pd.read_csv(TRAIN_PATH)
            self.df['ground_truth'] = self.df['answer']
            self.df['answer'] = -1
            
            if self.randomlize:
                self.df = self.df.reset_index().sample(frac=1).reset_index(drop=True)
            
            self.predict_called = True
            self.counter = 0
            self.len = len(self.df)
        
        
        def iter_test(self):
             while self.counter<self.len:
                if self.predict_called:
                    self.predict_called = False
                    yield (self.df.loc[[self.counter]][['id','problem']]),(self.df.loc[[self.counter]][['id','answer']])
                else:
                    print("You must call `predict()` successfully before you can continue with `iter_test()`")
                    yield None 
                
        def predict(self, answer):
            self.df.loc[self.counter, ('answer')] = answer['answer'].values[0]
            self.predict_called = True
            self.counter+=1

    env = train_env(randomize=True)
    iter_test = env.iter_test()
else:
    # Set up the evaluation API
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

TO-DO

Change temperature as the question goes longer
Change temperature based on question lenght

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [4]:
DEBUG = False
# QUANT = False

In [5]:
# %%capture
# %set_env CONDA_PREFIX=/opt/conda
# !pip install -U --no-index /kaggle/input/uv-package-manager/uv-0.2.2-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !uv pip uninstall torch
# !uv pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

In [6]:
# !pip uninstall torch
# !pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm
# !pip install --no-index -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install --no-index -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl --find-links /kaggle/input/vllm-whl

In [7]:
MODEL_NAME = '/scratch/aakash_ks.iitr/models/deepseek/'
#MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [8]:
stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"]  # this is only useful for code and not cot
#stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output", '\n\n', '<|eot_id|>',]

In [9]:
from vllm import LLM, SamplingParams

llm = LLM(
    model=MODEL_NAME,
    dtype="half",
    #enforce_eager=True, #to disable CUDA graphs for debugging 
    gpu_memory_utilization=0.93,  #with enforce eager you can use 0.99; else 0.91
    swap_space=2,  #CPU RAM per gpu preferable is 2, default is 4
    max_model_len=TOTAL_TOKENS,
    kv_cache_dtype="fp8",  #k: fp8_e5m2
    tensor_parallel_size=2,
    max_num_seqs=15, #default is 256 keep 20 while batching
    max_seq_len_to_capture=TOTAL_TOKENS, #k: context
    #disable_custom_all_reduce=True, #k: uncomment when using 0.4.3
)

# if getting CUDA OOM switch to using enforce_eager = True

WARNING 06-10 19:50:29 config.py:1086] Casting torch.bfloat16 to torch.float16.
INFO 06-10 19:50:29 config.py:379] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop without scaling factors. FP8_E5M2 (without scaling) is only supported on cuda version greater than 11.8. On ROCm (AMD GPU), FP8_E4M3 is instead supported for common inference criteria.


2024-06-10 19:50:38,931	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-10 19:50:40 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/scratch/aakash_ks.iitr/models/deepseek/', speculative_config=None, tokenizer='/scratch/aakash_ks.iitr/models/deepseek/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=fp8, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/scratch/aakash_ks.iitr/models/deepseek/)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-10 19:50:47 utils.py:660] Found nccl from library /home/aakash_ks.iitr/.config/vllm/nccl/cu11/libnccl.so.2.18.1
(RayWorkerWrapper pid=20564) INFO 06-10 19:50:47 utils.py:660] Found nccl from library /home/aakash_ks.iitr/.config/vllm/nccl/cu11/libnccl.so.2.18.1
INFO 06-10 19:50:51 selector.py:69] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-10 19:50:51 selector.py:32] Using XFormers backend.
(RayWorkerWrapper pid=20564) INFO 06-10 19:50:51 selector.py:69] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
(RayWorkerWrapper pid=20564) INFO 06-10 19:50:51 selector.py:32] Using XFormers backend.
INFO 06-10 19:50:56 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=20564) INFO 06-10 19:50:56 pynccl_utils.py:43] vLLM is using nccl==2.18.1
INFO 06-10 19:50:56 utils.py:132] reading GPU P2P access cache from /home/aakash_ks.iitr/.config/vllm/gpu_p2p_access_cache_for_0,1.json
(RayWorkerWrapper pid=20564) INFO 06-10 19:50:56 utils.p

In [10]:
import gc
import torch
# torch.backends.cuda.enable_mem_efficient_sdp(False)

In [11]:
import pandas as pd
from tqdm import tqdm
import math
import random

(RayWorkerWrapper pid=20564) INFO 06-10 19:51:20 custom_all_reduce.py:246] Registering 305 cuda graph addresses
(RayWorkerWrapper pid=20564) INFO 06-10 19:51:20 model_runner.py:1017] Graph capturing finished in 1 secs.


In [12]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

In [45]:
import re
import sys
import subprocess

def return_last_print(output, n):
    lines = output.strip().split('\n')
    if lines:
        return lines[n]
    else:
        return ""

def process_code(code, return_shell_output=False):
    
    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ', real=True)')
        else:
            return "{}{}".format(match.group()[:-1], ')')
    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)
    
    if not return_shell_output:
        print(code)
    with open('code.py', 'w') as fout:
        fout.write(code)
    
    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value=='FAIL':
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value+='\nTry checking the formatting and imports'
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS  
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e,'shell_output')
        code_output = -1
    
    if return_shell_output:
        if code_output==-1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS  
    
    
    return code_output


def process_text_output(output):
    result = output    
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED FINAL', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING TEXT')
        result_output = -1
    
    return result_output


In [14]:
code = """Below is a math problem you are to solve (non-negative integer answer!):
\"{}\"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""


cot = """Below is a math problem you are to solve (non-negative integer answer!):
\"{}\"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

promplt_options = [code,cot]

In [79]:
tokenizer = llm.get_tokenizer()
tokenizer.pad_token = tokenizer.eos_token

In [84]:
model_inputs = tokenizer(['hello world', 'wljkba oahfb kgakslhdfj iugaslfid iuhiuagsdfh', 'wow'], padding=True, truncation=True, return_tensors='pt')

In [89]:
len(model_inputs['input_ids'])

3

torch.Size([3, 24])

In [16]:
import re
from collections import defaultdict
from collections import Counter

from numpy.random import choice
import numpy as np
from copy import deepcopy

tool_instruction = '\n\nPlease integrate natural language reasoning with programs to solve the above problem, and put your final numerical answer within \\boxed{}.\nNote that the intermediary calculations may be real numbers, but the final numerical answer would always be an integer.'


#tool_instruction = " The answer should be given as a non-negative modulo 1000."
#tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

temperature = TEMP
top_p = 1.0

temperature_coding = TEMP_CODE
top_p_coding = 1.0

   
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2,3)


In [17]:
test, sample = next(iter_test)
print(test)

         id                                            problem
0  r02fc4ad  Jenna is at a fair with four friends. They all...


In [19]:
problem = test['problem'].values[0]
problem

'Jenna is at a fair with four friends. They all want to ride the roller coaster, but only three people can fit in a car. How many different groups of three can the five of them make?'

In [22]:
best, best_count = best_stats.get(0,(-1,-1))
best, best_count

(-1, -1)

In [23]:
i = 0

outputs = total_outputs.get(i,[])
text_answers, code_answers = question_type_counts.get(i,starting_counts)
results = total_results.get(i,[])
answers = total_answers.get(i,[])

In [24]:
cot_prompt = f"User: {cot.format(problem, '{}')}"
code_prompt = f"User: {code.format(problem, '{}')}"

prompts = [cot_prompt] * 9 + [code_prompt] * 6
current_printed = list(map(len, prompts))

input_len2 = deepcopy(current_printed)

In [25]:
prompts

['User: Below is a math problem you are to solve (non-negative integer answer!):\n"Jenna is at a fair with four friends. They all want to ride the roller coaster, but only three people can fit in a car. How many different groups of three can the five of them make?"\nAnalyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n',
 'User: Below is a math problem you are to solve (non-negative integer answer!):\n"Jenna is at a fair with four friends. They all want to ride the roller coaster, but only three people can fit in a car. How many different groups of three can the five of them make?"\nAnalyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n',
 'User: Below is a math problem you are to solve (non-negative integer answer!):\n"Jenna is at a fair with four friends. They all want

In [24]:
from random import shuffle

lst = ['hi', 'hellop', 'ok', 'bye']
shuffle(lst)

lst

['hellop', 'ok', 'bye', 'hi']

In [27]:
current_printed, input_len2

([423, 423, 423, 423, 423, 423, 423, 423, 423, 764, 764, 764, 764, 764, 764],
 [423, 423, 423, 423, 423, 423, 423, 423, 423, 764, 764, 764, 764, 764, 764])

In [28]:
generation_outputs = llm.generate(prompts, SamplingParams(
    stop=stop_words, temperature=temperature, max_tokens=TOTAL_TOKENS, top_p=top_p, 
    include_stop_str_in_output=True))

decoded_outputs = [prompt + output.outputs[0].text for prompt, output in zip(prompts, generation_outputs)]

Processed prompts: 100%|██████████| 15/15 [00:04<00:00,  3.13it/s]


In [29]:
decoded_outputs

['User: Below is a math problem you are to solve (non-negative integer answer!):\n"Jenna is at a fair with four friends. They all want to ride the roller coaster, but only three people can fit in a car. How many different groups of three can the five of them make?"\nAnalyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\nTo solve this problem, we can use the combination formula from combinatorics, which states that the number of ways to choose $k$ items from a set of $n$ items is given by $\\binom{n}{k}=\\frac{n!}{k!(n-k)!}$, where $n!$ denotes the factorial of $n$ (the product of all positive integers up to $n$).\n\nIn this case, we want to choose 3 friends out of 5, so we have $n=5$ and $k=3$. Plugging these values into the combination formula gives us:\n$$\\binom{5}{3}=\\frac{5!}{3!(5-3)!}=\\frac{5!}{3!2!}=\\frac{5\\times4}{2\\times1}=10.$$\nSo the five friends can make 10 diff

In [72]:
l = [1,2,3,3,4,5,6,7]
l.pop(0)
l

[2, 3, 3, 4, 5, 6, 7]

In [30]:
for k in range(15):
    current_printed[k] += len(decoded_outputs[k][current_printed[k]:])
    
current_printed  

[1050,
 948,
 827,
 977,
 637,
 891,
 859,
 964,
 746,
 1001,
 1247,
 1169,
 1403,
 1519,
 1124]

In [32]:
stop_word_cond = [False] * 15
for stop_word in stop_words:
    for k in range(15):
        stop_word_cond[k] = stop_word_cond[k] or (decoded_outputs[k][-len(stop_word):]==stop_word)
        
print(stop_word_cond)

[False, True, True, True, True, True, True, False, True, True, True, True, True, True, True]


In [40]:
any([False, False])

False

In [41]:
new_prompts = [""] * 15


In [49]:
code_outputs = [-1] * 15


In [ ]:
for k in range(15): 
    
    if not stop_word_cond[k]:
        continue
                   
    if (decoded_outputs[k][-len("```python"):]=="```python"):
        temperature_inner=temperature_coding
        top_p_inner = top_p_coding
        new_prompts[k] = decoded_outputs[k]
    else:
        temperature_inner=temperature
        top_p_inner = top_p
        try:
            if (decoded_outputs[k][-len("``````output"):]=="``````output"):
                code_text = decoded_outputs[k].split('```python')[-1].split("``````")[0]
            else:
                code_text = decoded_outputs[k].split('```python')[-1].split("```")[0]
            
            code_output, CODE_STATUS = process_code(code_text, return_shell_output=True)
            print('CODE RESULTS', code_output)
            
            if not CODE_STATUS:
                continue
                
        except Exception as e:
            print(e)
            print('ERROR PARSING CODE')
            code_output = -1
            
        if code_output!=-1:
            if (decoded_outputs[k][-len(")\n```"):]==")\n```"):
                new_prompts[k] = decoded_outputs[k]+'```output\n'+str(code_output)+'\n```\n'
            else:
                new_prompts[k] = decoded_outputs[k]+'\n'+str(code_output)+'\n```\n'

        else:
            continue
        
        code_outputs[k] = code_output

In [50]:
new_prompts

['',
 'User: Below is a math problem you are to solve (non-negative integer answer!):\n"Jenna is at a fair with four friends. They all want to ride the roller coaster, but only three people can fit in a car. How many different groups of three can the five of them make?"\nAnalyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\nThis problem involves the concept of combinations in mathematics. A combination is a selection of items without considering the order. The number of combinations of $n$ items taken $k$ at a time is given by the formula $\\binom{n}{k} = \\frac{n!}{k!(n-k)!}$, where $n!$ denotes the factorial of $n$ (the product of all positive integers up to $n$).\n\nIn this case, we have 5 friends (Jenna and her 4 friends) and we want to choose 3 of them to ride the roller coaster together. So we need to calculate $\\binom{5}{3}$.\n\n```python',
 'User: Below is a math proble

In [51]:
code_outputs

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [57]:
new_prompts = [prompt for prompt in new_prompts if prompt]

generation_output = llm.generate(new_prompts, SamplingParams(
    stop=stop_words, temperature=temperature_inner, max_tokens=TOTAL_TOKENS, top_p=top_p_inner,
    include_stop_str_in_output=True))

decoded_outputs = [prompt + output.outputs[0].text for prompt, output in zip(new_prompts, generation_output)]

Processed prompts: 100%|██████████| 13/13 [00:02<00:00,  4.77it/s]


In [58]:
decoded_outputs

['User: Below is a math problem you are to solve (non-negative integer answer!):\n"Jenna is at a fair with four friends. They all want to ride the roller coaster, but only three people can fit in a car. How many different groups of three can the five of them make?"\nAnalyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\nThis problem involves the concept of combinations in mathematics. A combination is a selection of items without considering the order. The number of combinations of $n$ items taken $k$ at a time is given by the formula $\\binom{n}{k} = \\frac{n!}{k!(n-k)!}$, where $n!$ denotes the factorial of $n$ (the product of all positive integers up to $n$).\n\nIn this case, we have 5 friends (Jenna and her 4 friends) and we want to choose 3 of them to ride the roller coaster together. So we need to calculate $\\binom{5}{3}$.\n\n```python\nfrom math import factorial\n\ndef ca

In [60]:
for stop_word in stop_words:
    for k in range(15):
        stop_word_cond[k] = stop_word_cond[k] or (decoded_outputs[k][-len(stop_word):]==stop_word)
        
print(stop_word_cond)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [61]:
for k in range(13): 
    
    if not stop_word_cond[k]:
        continue
                   
    if (decoded_outputs[k][-len("```python"):]=="```python"):
        temperature_inner=temperature_coding
        top_p_inner = top_p_coding
        new_prompts[k] = decoded_outputs[k]
    else:
        temperature_inner=temperature
        top_p_inner = top_p
        try:
            if (decoded_outputs[k][-len("``````output"):]=="``````output"):
                code_text = decoded_outputs[k].split('```python')[-1].split("``````")[0]
            else:
                code_text = decoded_outputs[k].split('```python')[-1].split("```")[0]
            
            code_output, CODE_STATUS = process_code(code_text, return_shell_output=True)
            print('CODE RESULTS', code_output)
            
            if not CODE_STATUS:
                continue
                
        except Exception as e:
            print(e)
            print('ERROR PARSING CODE')
            code_output = -1
            
        if code_output!=-1:
            if (decoded_outputs[k][-len(")\n```"):]==")\n```"):
                new_prompts[k] = decoded_outputs[k]+'```output\n'+str(code_output)+'\n```\n'
            else:
                new_prompts[k] = decoded_outputs[k]+'\n'+str(code_output)+'\n```\n'

        else:
            continue
        
        code_outputs[k] = code_output

10.0

CODE RESULTS 10.0
10.0

CODE RESULTS 10.0
10.0

CODE RESULTS 10.0
10

CODE RESULTS 10
10.0

CODE RESULTS 10.0
10

CODE RESULTS 10
10.0

CODE RESULTS 10.0
10

CODE RESULTS 10
10

CODE RESULTS 10
10

CODE RESULTS 10
10

CODE RESULTS 10
10

CODE RESULTS 10
10

CODE RESULTS 10


In [62]:
code_outputs

['10.0',
 '10.0',
 '10.0',
 '10',
 '10.0',
 '10',
 '10.0',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 -1,
 -1]

In [ ]:
for i, (test, sample_submission) in tqdm(enumerate(iter_test)):
    print(f"Solving problem {i} ...")
    TIME_SPENT = time.time() - NOTEBOOK_START_TIME

    if TIME_SPENT>TIME_LIMIT:
        sample_submission['answer'] = 0
        env.predict(sample_submission)
        break
    
    Q_START_TIME = time.time()
    
    problem = test['problem'].values[0]
    
    for jj in range(3):
        print(f"\n\n\nQUESTION {i} - {jj} - TIME_SPENT : {TIME_SPENT:.0f} secs")
        
        best, best_count = best_stats.get(i,(-1,-1))
        if best_count>BEST_COUNT_THRESHOLD:      # jj instead of n_repetitions
            print("SKIPPING CAUSE FOUND BEST")
            break
        
        time_now = time.time()
        
        if (time_now - Q_START_TIME) > PER_Q_TIME_LIMIT or (time_now - NOTEBOOK_START_TIME)>TIME_LIMIT-200:
            print(f'BREAKING BECAUSE QUESTION TIME LIMIT EXCEEDED')
            break
        
        outputs = total_outputs.get(i,[])
        text_answers, code_answers = question_type_counts.get(i,starting_counts)
        results = total_results.get(i,[])
        answers = total_answers.get(i,[])

        try:
            #initail_message = problem  + tool_instruction 
           
            cot_prompt = f"User: {cot.format(problem, '{}')}"
            code_prompt = f"User: {code.format(problem, '{}')}"
            
            prompts = [cot_prompt] * 9 + [code_prompt] * 6
            current_printed = list(map(len, prompts))

            input_len2 = deepcopy(current_printed)

            generation_outputs = llm.generate(prompts, SamplingParams(
                stop=stop_words, temperature=temperature, max_tokens=TOTAL_TOKENS, top_p=top_p, 
                include_stop_str_in_output=True))
            
            decoded_outputs = [prompt + output.outputs[0].text for prompt, output in zip(prompts, generation_outputs)]
    
            for k in range(15):
                print(f"\n\nOutput {k}:\n{decoded_output[k][current_printed:]}\n")
                current_printed[k] += len(decoded_output[k][current_printed[k]:])           
            
            stop_word_cond = [False] * 15
            for stop_word in stop_words:
                for k in range(15):
                    stop_word_cond[k] = stop_word_cond[k] or (decoded_output[k][-len(stop_word):]==stop_word)
                    
            print(stop_word_cond)
            
            while any(stop_word_cond):
                new_prompt = [""] * 15
                
                for k in range(15):                
                    if (decoded_output[k][-len("```python"):]=="```python"):
                        temperature_inner=temperature_coding
                        top_p_inner = top_p_coding
                        new_prompt[k] = decoded_output[k]
                    else:
                        temperature_inner=temperature
                        top_p_inner = top_p
                        try:
                            if (decoded_output[k][-len("``````output"):]=="``````output"):
                                code_text = decoded_output[k].split('```python')[-1].split("``````")[0]
                            else:
                                code_text = decoded_output[k].split('```python')[-1].split("```")[0]
                            
                            code_output, CODE_STATUS = process_code(code_text, return_shell_output=True)
                            print('CODE RESULTS', code_output)
                            
                            if not CODE_STATUS:
                                continue
                                
                        except Exception as e:
                            print(e)
                            print('ERROR PARSING CODE')
                            code_output = -1
                            
                        if code_output!=-1:
                            if (decoded_output[k][-len(")\n```"):]==")\n```"):
                                new_prompt[k] = decoded_output[k]+'```output\n'+str(code_output)+'\n```\n'
                            else:
                                new_prompt[k] = decoded_output[k]+'\n'+str(code_output)+'\n```\n'

                        else:
                            continue


                generation_output = llm.generate(new_prompt, SamplingParams(
                    stop=stop_words, temperature=temperature_inner, max_tokens=TOTAL_TOKENS, top_p=top_p_inner,
                    include_stop_str_in_output=True))
                
                generation_output = list(map(lambda x: x.outputs[0].text, generation_output))
                
                decoded_output = list(map(lambda p, o: p + o, zip(new_prompt, generation_output)))

                result_output = []
                code_output = []
                
                for k in range(15):
                    print(f"\nINTERMEDIATE OUT {k}:\n{decoded_output[k][current_printed:]}\n")
                    current_printed[k]+=len(decoded_output[k][current_printed[k]:])

            
                    raw_output = decoded_output[k][input_len2[k]:]
            
                    #print(f"\n\nOutput :\n{raw_output}\n")                            
                    result_output.append(process_text_output(raw_output))
            
                    try:
                        code_output[k] = round(float(eval(code_output[k]))) % 1000
                        
                    except Exception as e:
                        print(e,'final_eval')
                        code_output = -1
                        

        except Exception as e:
            print(e,"5")
            result_output, code_output = -1, -1

        if code_output!=-1:
            outputs.append(code_output)
            code_answers+=1

        if result_output!=-1:
            outputs.append(result_output)
            text_answers+=1

        if len(outputs) > 0:
            occurances = Counter(outputs).most_common()
            print(occurances)
            if occurances[0][1] > best_count:
                print("GOOD ANSWER UPDATED!")
                best = occurances[0][0]
                best_count = occurances[0][1]
            if occurances[0][1] > BEST_COUNT_THRESHOLD:
                print("ANSWER FOUND!")
                break

        results.append(result_output)
        answers.append(code_output)
        
        best_stats[i] = (best, best_count) 
        question_type_counts[i] = (text_answers, code_answers)
        total_outputs[i] = outputs
        
        total_results[i] = results
        total_answers[i] = answers

        print("code_answers",code_answers-starting_counts[1],"text_answers",text_answers-starting_counts[0])
        if DEBUG:
            break
            
    print(f"Predicted best answer: {best_stats}")
    sample_submission['answer'] = best_stats[i][0]
    env.predict(sample_submission)
    
    print('-' * 80)
    print(f'Time spent on the question: {time.time() - Q_START_TIME:.0f} secs')
    print('-' * 80)

In [ ]:
if not PRIVATE:
    display(env.df)
    print(f"\ncorrect: {(env.df['ground_truth'] == env.df['answer']).sum()} out of {len(env.df)}")

In [ ]:
with open('code.py', 'w') as fout:
    fout.write("print('done')")

batcmd = 'timeout 5 ' + sys.executable + ' code.py'
try:
    shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
    print(shell_output)
except:
    pass

In [ ]:
time.time() - NOTEBOOK_START_TIME